In [1]:
from keras.models import load_model
import os
import keras.backend.tensorflow_backend as tb
tb._SYMBOLIC_SCOPE.value = True
os.environ['KMP_DUPLICATE_LIB_OK']='True'


import time
import json
import mysql.connector
from SocialProcessor import SocialProcessor

import vk_api
import threading

%matplotlib inline

Using TensorFlow backend.


In [2]:
class StoppableThread(threading.Thread):
    """
    Thread class with a stop() method. The thread itself has to check
    regularly for the stopped() condition.
    
    """
    func = None
    args = {}

    def __init__(self,  *args, **kwargs):
        super(StoppableThread, self).__init__(*args, **kwargs)
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def stopped(self):
        return self._stop_event.is_set()
    
    def run(self):
        self.args['thread'] = self
        self.func(**self.args)
    

In [3]:
model_path = 'keras-facenet-master/model/keras/model/facenet_keras.h5'
model = load_model(model_path)

/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
def aunt_vk():
    return input(), True


def capt_vk(captcha):
    print(captcha.get_url())
    inp = input()
    captcha.try_again(inp)

In [5]:
credentials = {}
with open('credentials.json', 'r') as file:
    credentials = json.load(file)

In [6]:
vk_session = vk_api.VkApi(credentials['login'], credentials['password'], auth_handler=aunt_vk, captcha_handler=capt_vk)
vk_session.auth()

api = vk_session.get_api()

In [7]:
def performVkProcessing(delay=0, count=100, thread=False):
    conn = mysql.connector.connect(user='face', host="localhost", password="Hr3mI3PM5JlFWrub", database='global_scrape', port=8889, collation="utf8mb4_general_ci")
    processor = SocialProcessor(conn, model, prefix='/Users/aleksandrdremov/PycharmProjects/GlobalScraper/', batch=512)
    time.sleep(delay)
    while count>0:
        if type(thread) != type(False):
            if thread.stopped():
                break
        task = processor.getTask()
        user_id = task[0]
        query_id = task[-1]
        processor.delTask(query_id)
        
        processor.processVkUser(api, user_id)
        count-=1

In [8]:
# performVkProcessing(0,1)

In [9]:
threads = []
threading.active_count()

5

In [17]:
def addThreads(current_num, max_num=20):
    t_num = max(0, max_num - current_num)
    limits = 1
    step = 1.5
    params = [(i*step, limits) for i in range(t_num)]
    print("Added %s threads"% (len(params)))
    for args in params:
        x = StoppableThread(limits=800)
        x.func = performVkProcessing
        args = {"delay": args[0], "count": args[1]}
        x.args = args
        x.start()
        threads.append(x)

168265890:


In [11]:
while True:
    active = sum([i.isAlive() for i in threads])
    print("Active threads:", active)
    addThreads(active)
    active = sum([i.isAlive() for i in threads])
    print("Active threads:", active)
    i = 0
    while i < len(threads):
        if threads[i].isAlive():
            i += 1
        else:
            threads.pop(i)
    time.sleep(30)

Active threads: 0
Added 20 threads
Active threads: 20
Procesing 77689734


Procesing 72944536


Procesing 185313244


Procesing 448222583


Procesing 174533008


...

Procesing 367750982




367750982:
Added 0 rows
12662705:
Added 6 rows

301054564:
Added 4 rows


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-b965d3415cf8>", line 13, in <module>
    time.sleep(30)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2044, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/aleksandrdremov/anaconda3/envs/FaceScrape/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1169, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_o


KeyboardInterrupt



In [22]:
sum([i.isAlive() for i in threads])

13

112634373:
Added 14 rows

153007409:

Added 50 rows

274377358:
179437977:
Added 29 rows
Added 67 rows


In [14]:
conn = mysql.connector.connect(user='face', host="localhost", password="Hr3mI3PM5JlFWrub", database='global_scrape', port=8889, collation="utf8mb4_general_ci")
processor = SocialProcessor(conn, model, prefix='/Users/aleksandrdremov/PycharmProjects/GlobalScraper/', batch=512)
# processor.processVkUser(api, 1422)

381483022:
